## 必要なライブラリのimport

In [ ]:
# 外部パッケージのインストール
# !pip3 install netcdf4 
# !pip3 install --user https://github.com/matplotlib/basemap/archive/master.zip

import csv
import re
from pathlib import Path
import conv_NC4files as cnc
import pandas as pd
import numpy as np

## ディレクトリの設定

動作確認のncファイルは

`S5P_OFFL_L2__SO2____20190806T022006_20190806T040136_09388_01_010107_20190812T122912.nc`

を使用した。

In [ ]:
# 設定
INDIR  = './nc'  # ncファイルの格納先
OUTDIR = './out'   # 中間ファイルの格納先

## .ncファイルの読み込みと変換

In [ ]:
# 設定をパスに変換
in_path = Path(INDIR)
out_path = Path(OUTDIR)

# CSV出力先のディレクトリを作成
csv_path = out_path.joinpath('SO2/nc4csv00')
if not csv_path.exists():
    csv_path.mkdir(parents=True)

# ncファイルをCSVファイルに変換
optkeys = ('sulfurdioxide_total_vertical_column',
           'sulfurdioxide_total_vertical_column_precision')
for nc_file in in_path.glob('*.nc'):
    print('convert:', nc_file)
    csv_file = csv_path.joinpath('nc4csv00_'+nc_file.stem+'.csv')
    cnc.conv_s5p_csv(nc_file, csv_file, optkeys)

print('done')

## 空行の削除と値のフィルタリング

In [ ]:
# フィルタリングしたファイルの出力先ディレクトリを作成
filtered_csv_path = out_path.joinpath('SO2/nc4csv00_filtered')
if not filtered_csv_path.exists():
    filtered_csv_path.mkdir(parents=True)

# フィルタリング
for csv_file in csv_path.glob('*.csv'):
    print('filtering: ', csv_file)

    # pandasで元のCSVファイルを読み込み
    df = pd.read_csv(csv_file)
   
    # 偶数行は空なので奇数行のみ抽出
    df = df[1::2]
    
    # そのままでは精度が高すぎるため、qa_value 0.75より大きい値を
    # 抽出してフィルタリングする
    df = df[df['qa_value'] > 0.5]
   
    # 元のファイル名のsuffixを取り出し（prefixを削除）
    suffix = re.sub(r'^.*____', '', csv_file.stem)
    # 変換後の出力ファイルパスは、CSV出力先ディレクトリ下にしておく
    outfile = filtered_csv_path.joinpath('SO2_out_scr_'+suffix+'.csv')
    df.to_csv(outfile)

print('done')

## 緯度、経度、SO2カラム量の抽出

In [ ]:
lon_total = []
lat_total = []
SO2_total = []
#SO2の単位はDU(Dobson Unit)
#DU = 2241.15
mol = 6.00214*1.e+4

for file in filtered_csv_path.glob('*.csv'):
    print('read:', file)
    d = np.loadtxt(file, delimiter=',', skiprows=1, unpack=True)
    lon = d[2,:]
    lat = d[3,:]
    SO2 = mol * d[5,:]
    lon_total.extend(lon)
    lat_total.extend(lat)
    SO2_total.extend(SO2)
    
print('done')

## Plotに必要なライブラリのimport

In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

## 地図の設定

In [ ]:
# Basemap instanceの作成、緯度経度を西之島に合わせる
# 緯度経度の範囲は自分の好きなようにカスタマイズする
m = Basemap(projection='merc',
            resolution='h',
            llcrnrlon=135,
            llcrnrlat=25,
            urcrnrlon=155,
            urcrnrlat=37)

## plotとSO2濃度の調整

In [ ]:
plt.figure(figsize=(10,10))
plt.title('yyyy/mm/dd')

#海岸線を描く
m.drawcoastlines()

lon, lat = m(lon_total, lat_total)
#火山や山火事などSO2が大量に放出する場合vmax=20にするとSO2濃度が鮮明になる
im = m.scatter(lon, lat, alpha=0.9, s=1, vmin=0., vmax=2.e+1,
               c=SO2_total, edgecolors='none', linewidths=1.5,
               cmap="jet")

cbar = m.colorbar(im, location='bottom', pad="5%")
cbar.set_label('$10^{15}molec/cm^2$')

plt.show()